# 共同研究者を管理する

GRDMプロジェクトの共同研究者の追加や削除、権限の管理を行う。

## メニューバー

In [ ]:
# TODO: Implement the menu bar

##  1. フォームセクション

In [ ]:
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.common.manage_collaborators import CollaboratorManager
CollaboratorManager(working_path=os.path.abspath('__file__')).generateFormScetion()

## 2. メイン処理セクション

### 2-1. 設定反映処理

In [ ]:
# TODO: Implement

### 2-2. タスク完了処理

In [ ]:
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.common.manage_collaborators import CollaboratorManager
CollaboratorManager(working_path=os.path.abspath('__file__')).completed_task()

## サブフローメニューへ

サブフローメニューへアクセスするボタンを表示します

In [ ]:
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.common.manage_collaborators import CollaboratorManager
CollaboratorManager(working_path=os.path.abspath('__file__')).return_subflow_menu()